In [1]:
#pip install -U imbalanced-learn


In [2]:
import imblearn
import datetime

In [3]:
import pandas as pd
import seaborn as sns
import cv2
import numpy as np

from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

import tensorflow as tf
# import tensorflow.compat.v1 as tf
# from tensorflow import keras
from tensorflow.keras.layers import Lambda
from keras.preprocessing.sequence import pad_sequences
import os, sys, time
from math import floor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from collections import OrderedDict
# from tensorflow.python.ops import math_ops
# import tensorflow.keras.backend as kb
from tensorflow_addons.utils.ensure_tf_install import _check_tf_version
_check_tf_version()
from tensorflow_addons import losses
# Custom Functions
# Added to path by using sys.path.append('')
# sys.path.append('/home/akanu/git/anomalous_pred/custom_functions')
# sys.path.insert(0,'/home/akanu/git/anomalous_pred/custom_functions')
# sys.path.append('/mnt/roahm/users/akanu/git/anomalous_pred/custom_functions')
from load_data import Files_Load, Boxes, test_split_norm_abnorm, norm_train_max_min
from custom_metrics import bb_intersection_over_union, bb_intersection_over_union_np
from coordinate_change import xywh_tlbr, tlbr_xywh
from pedsort import pedsort
from load_data_binary import *

from TP_TN_FP_FN import seperate_misclassifed_examples, sort_TP_TN_FP_FN_by_vid_n_frame
from plot_all_videos import cycle_through_videos

# Kmeans
from sklearn.cluster import KMeans

# Need to figue out why this is not working
# from persistence1d import RunPersistence


from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import plot_confusion_matrix

In [4]:
frames = 20

# train_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Train_Box/"
# test_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Test_Box/"

train_file = "/home/akanu/akanu/dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Train_Box"
test_file = "/home/akanu/akanu/dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Test_Box/"

# loc_files_train, loc_files_test, box_train_txt, box_test_txt = Files_Load(train_file, test_file)
loc = Files_Load(train_file, test_file)
# Don't forget to change to xywh 
traindict = Boxes(loc['files_train'], loc['txt_train'], 
                      frames, pad ='pre', to_xywh=True)
testdict = Boxes(loc['files_test'], loc['txt_test'],
                 frames, pad ='pre', to_xywh = True)
abnormal_dict, normal_dict = test_split_norm_abnorm(testdict) #splits by which predicted frames are normal or not

# Normilize data
max1 = traindict['x_ppl_box'].max()
min1 = traindict['x_ppl_box'].min()
xx,yy = norm_train_max_min(data= traindict, max1=max1,min1=min1)
xx_norm,yy_norm = norm_train_max_min(data = normal_dict, max1=max1,min1=min1)
xx_abnorm,yy_abnorm = norm_train_max_min(data = abnormal_dict, max1=max1,min1=min1)


In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32
xx_train, xx_val,yy_train,yy_val = train_test_split(xx,yy, test_size = 0.3)
train_univariate = tf.data.Dataset.from_tensor_slices((xx_train,yy_train))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
val_univariate = tf.data.Dataset.from_tensor_slices((xx_val,yy_val))
val_univariate = val_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [6]:
model_loc = '/home/akanu/akanu/projects/anomalous_pred/results_all_datasets/experiment_1/saved_model'
loaded_model = os.path.join(model_loc,
                            '01_07_2021_lstm_network_xywh_avenue_20.h5')


lstm = tf.keras.models.load_model(loaded_model,  
                                   custom_objects = {'loss':'mse'} , 
                                   compile=True)

In [7]:
exp = { '1': True,
        '2': False,
        '3_1': False,
        '3_2': False,
        # want an adaptive model saved based on arch size for model_loc
        'data': 'hr-st', #st, avenue,hr-st
        }

hyparams = {
    'epochs':500,
    'batch_size': 32,
    'buffer_size': 10000,
 
    'frames': 20,

    'to_xywh':True, # This is assuming file is in tlbr format
    # 'max':913.0, # wonder better way to pick
    # 'min':-138.5, # wonder better way to pick

    'networks': {
        'lstm':{
            'loss':'mse',
            'lr': 8.726e-06,
            'early_stopping': True,
            'mointor':'loss',
            'min_delta': 0.00005,
            'patience': 15,
            'val_ratio': 0.3,
        },


        'binary_classifier':{
            'neurons': 30,
            'dropout':0.3,
            'lr': 0.0001, #0.00001
            'save_model': False,
            'early_stopping':False,
            'mointor': 'loss',
            'min_delta':0.0000005,
            'batch_size': 32,
            'patience':30,
            'seed':40,
            'abnormal_split':0.5, # Split for the data into normal and abnormal
            'val_ratio':0.3, #guarantee the ratio of normal and abnormal frames
                            # are the same for validatin set and training set.
                            # so val_ratio. Think val_ratio(normal) + val_ratio(abnormal ) = val_ratio(normal + abnormal)

        }


    }

}

name_exp = None
if exp['1']:
    name_exp = '1'
elif exp['2']:
    name_exp = '2'
elif exp['3_1']:
    name_exp ='3_1'
elif exp['3_2']:
    name_exp = '3_2'

date = datetime.datetime.now()


loc =  {
    # if I'm running a test where don't want to save anything
    # how do I do that. Maybe move them to tmp
    
    'model_path_list': ['results_all_datasets', 'experiment_{}'.format(name_exp), 'saved_model'],
    'metrics_path_list': ['results_all_datasets', 'experiment_{}'.format(name_exp), 'metrics_plot'], 

    'nc':{
        'model_name': 'lstm_network',
        'model_name_binary_classifer': 'binary_network',
        'data_coordinate_out': 'xywh',
        'dataset_name': exp['data'], # avenue, st                   #################
        # ________________________________________________________________________________________________________________________________--
        'date': date.strftime("%m") + '_' +date.strftime("%d") + '_' + date.strftime("%Y"),
        },
    # is nc the best way to propate and save things as same name
    # Might want to autmoically create a new folder with model arch saved
    # as a text file as well as in folder name

    'data_load':{
            'avenue':{
                # These are good because these locations are perm unless I manually move them
                'train_file': "/mnt/roahm/users/akanu/projects/anomalous_pred/output_deepsort/avenue/train_txt/",
                'test_file': "/mnt/roahm/users/akanu/projects/anomalous_pred/output_deepsort/avenue/test_txt/",
                },

            #Need to rerun ped1
            # 'ped1':{
            #     'train_file':"/mnt/roahm/users/akanu/dataset/Anomaly/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Txt_Data/Train_Box_ped1/",
            #     "test_file": "/mnt/roahm/users/akanu/dataset/Anomaly/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Txt_Data/Test_Box_ped1/",
            #     },
            'st':{
                'train_file':"/mnt/roahm/users/akanu/projects/anomalous_pred/output_deepsort/st/train_txt/",
                "test_file": "/mnt/roahm/users/akanu/projects/anomalous_pred/output_deepsort/st/test_txt/",
                },
            'hr-st':{
                'train_file':"/mnt/roahm/users/akanu/projects/anomalous_pred/output_deepsort/HR-ShanghaiTech/train_txt/",
                "test_file": "/mnt/roahm/users/akanu/projects/anomalous_pred/output_deepsort/HR-ShanghaiTech/test_txt/",
                },
            }


}

In [8]:
def data_binary(traindict, testdict, lstm, max1, min1):
    # I could have three main files that I load or
    # I could use an if statemet to switch bewteen the
    # experiments? What is the best method to use?
    # OR I could create another function that does loading?

    # Note that I am using the testing dable than Mac OS?
    # The answer is simple – more control to the user while providing betteta
    # If I do end up changing my normlization only need
    # to change in main file. Design intent


    # Test Data
    x,y = norm_train_max_min(   testdict,
                                # max1 = hyparams['max'],
                                # min1 = hyparams['min']
                                max1 = max1,
                                min1 = min1
                                )

    iou = compute_iou(x,y,lstm)
    
    # Note that indices returned are in same order
    # as testdict unshuffled

    # Note that indexing works only if data is loaded the same way
    # Every time . Otherwise I could create an lstm model then I would train it.
    # If loaded again then I would need to make


    ## Find indices
    indices = return_indices(   testdict['abnormal'],
                                seed = hyparams['networks']['binary_classifier']['seed'],
                                abnormal_split = hyparams['networks']['binary_classifier']['abnormal_split']
                                )


    # Gets the train and test data
    # returns a dict with keys: x, y
    train, test = binary_data_split(iou, indices)

    # Simple mapping
    if exp['1']:
        exp_1, exp_3 = True, False
    elif exp['2']:
        exp_1, exp_3 = False, False
    elif exp['3_1']:
        exp_1, exp_3 = True, True
    elif exp['3_2']:
        exp_1, exp_3 = False, True
    else:
        print("check experiment mapping ")
        quit()

    if exp_1:
        if exp_3:
            # Makes same amount of normal and abnormal in train
            train = reduce_train(   train['x'],
                                    train['y'],
                                    seed = hyparams['networks']['binary_classifier']['seed'])

            

        train, val = train_val_same_ratio(  train['x'],
                                            train['y'],
                                            hyparams['networks']['binary_classifier']['val_ratio'],
                                            seed = hyparams['networks']['binary_classifier']['seed']
                                            )


    else:
        x,y = norm_train_max_min(   traindict,
                                    # max1 = hyparams['max'],
                                    # min1 = hyparams['min']
                                    max1 = max1,
                                    min1 = min1
                                    )
        iou = compute_iou(x, y ,lstm)


        # Note training_set_from_lstm has iou in first column
        # second colum is the index( Put in a filler -1)
        training_set_from_lstm = np.append( iou.reshape((-1,1)),
                                            -np.ones((len(iou), 1)),
                                            axis=1
                                            )
        temp_combined_train = {}

        # print('train[x] {}'.format(train['x'].shape))
        # print('training from lstm set {}'.format(training_set_from_lstm.shape))
        temp_combined_train['x']= np.append(    train['x'],
                                                training_set_from_lstm,
                                                axis=0
                                                )

        # print('shape of tem[ combined from lstm set {}'.format(temp_combined_train['x'].shape))


        #since training set appended is coming from the orginal data
        # we know that all the labels are zeros because its normal

        temp_combined_train['y'] = np.append(train['y'],
                                                np.zeros(len(iou), dtype=np.int8)
                                                )

        if exp_3:
            temp_combined_train = reduce_train( temp_combined_train['x'],
                                                temp_combined_train['y'],
                                                seed = hyparams['networks']['binary_classifier']['seed']
                                                )

        train, val = train_val_same_ratio(  temp_combined_train['x'],
                                            temp_combined_train['y'],
                                            val_ratio=hyparams['networks']['binary_classifier']['val_ratio'],
                                            seed = hyparams['networks']['binary_classifier']['seed']
                                            )
        
    return train, val, test

In [11]:
train, val, test = data_binary(traindict, testdict, lstm, max1, min1)

neg, pos = np.bincount(train['y'])
print(pos/neg)
print("pos:{}, neg:{}".format(pos,neg)) 

0.01317732882958092
pos:1026, neg:77861


In [12]:
def tensorify(train, val, batch_size):

    """
    Mainly using this function to make training and validation sets
    train: dict that contains x and y
    val:dict that contains x and y

    return
    train_univariate: training tensor set
    val_univariate: validation tensor set
    """

    buffer_size = hyparams['buffer_size']
    batch_size = hyparams['batch_size']


    # print('Inside Tensorfify')
    """
    Come back and fix, I dont like tha I need to cast to float32
    """

    print(train['x'].shape)
    print(train['y'].shape)
    print(val['x'].shape)
    print(val['y'].shape)
    train_univariate = tf.data.Dataset.from_tensor_slices((train['x'], np.array(train['y'], dtype=np.float32)))
    train_univariate = train_univariate.cache().shuffle(buffer_size).batch(batch_size)
    val_univariate = tf.data.Dataset.from_tensor_slices((val['x'],np.array(val['y'], dtype=np.float32)))
    val_univariate = val_univariate.cache().shuffle(buffer_size).batch(batch_size)

    return train_univariate, val_univariate


In [13]:
    
    # Removing the index so I can pass into tensofify and not have two
    # functions that do similar things.
    train_no_index, val_no_index = {},{}
    val_no_index['x'] = val['x'][:,0]
    val_no_index['y'] = val['y']
 
    train_no_index['x'] = train['x'][:,0]
    train_no_index['y'] = train['y']


    train_tensor, val_tensor = tensorify(
                                            train_no_index,
                                            val_no_index,
                                            batch_size = 32)


(78887,)
(78887,)
(33807,)
(33807,)


In [16]:
for i in train_tensor:
    print(i)

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.34778965, 0.35161213, 0.29474064, 0.52387155, 0.31004162,
       0.39082416, 0.302396  , 0.35180181, 0.29328384, 0.57533977,
       0.31982684, 0.3649462 , 0.33472964, 0.29043608, 0.37753959,
       0.27129366, 0.35425277, 0.30290186, 0.30140991, 0.30469338,
       0.3989796 , 0.60446535, 0.36602509, 0.44177784, 0.30851095,
       0.28906824, 0.31792167, 0.31538316, 0.27531983, 0.32036889,
       0.2958624 , 0.33208034])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.29043309, 0.05393996, 0.29511319, 0.3734845 , 0.32335333,
       0.29690585, 0.26219659, 0.30133527, 0.29164615, 0.32678483,
       0.28263167, 0.31608828, 0.37968054, 0.29824272, 0.28368596,
       0.30903435, 0.36532301, 0.2964902 , 0.30770355,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.33537749, 0.32644859, 0.35341856, 0.42267022, 0.35378547,
       0.2985595 , 0.28250785, 0.30937715, 0.30509747, 0.30588366,
       0.31203446, 0.30979041, 0.38375588, 0.29743233, 0.35597484,
       0.32828549, 0.30774636, 0.379395  , 0.29499968, 0.28943388,
       0.15560334, 0.28248336, 0.3000298 , 0.31089531, 0.31366653,
       0.28054192, 0.28988814, 0.30070634, 0.29590963, 0.32886095,
       0.31992627, 0.3373087 ])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.30461303, 0.28849745, 0.30935046, 0.19675413, 0.28250707,
       0.31540904, 0.31036463, 0.2906094 , 0.32966651, 0.28943861,
       0.28565511, 0.28660319, 0.33027654, 0.21944332, 0.30155208,
       0.27136816, 0.28985624, 0.27614844, 0.29847632,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.55860414, 0.61412871, 0.29940458, 0.37752872, 0.32083646,
       0.3071169 , 0.28958575, 0.32457543, 0.30569365, 0.30406305,
       0.30800039, 0.29547388, 0.33032158, 0.30217099, 0.39363053,
       0.36352733, 0.31123227, 0.32205905, 0.30167098, 0.31380187,
       0.38155696, 0.30024014, 0.27204778, 0.35815174, 0.33120361,
       0.38056101, 0.42832345, 0.37564707, 0.28814153, 0.3105373 ,
       0.35801314, 0.60821087])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.32373718, 0.24330578, 0.20448471, 0.34755817, 0.30826423,
       0.34925605, 0.40422257, 0.25163918, 0.30400675, 0.37228997,
       0.36493183, 0.33438397, 0.5275218 , 0.29295516, 0.        ,
       0.31588536, 0.41076252, 0.31505849, 0.36870521,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.31757994, 0.34875284, 0.28185111, 0.30025238, 0.295087  ,
       0.31249466, 0.28306739, 0.29918383, 0.30155523, 0.33471672,
       0.3089298 , 0.30966733, 0.30345736, 0.30811753, 0.32240808,
       0.35118566, 0.30696307, 0.29027343, 0.32614072, 0.30488835,
       0.25837598, 0.32123364, 0.31445877, 0.26274568, 0.33494373,
       0.31494345, 0.29805328, 0.34568287, 0.32220964, 0.30745057,
       0.11615766, 0.31336524])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.29512874, 0.35059478, 0.28920144, 0.38038446, 0.28419871,
       0.2975552 , 0.29884508, 0.27451457, 0.3527172 , 0.31327209,
       0.29038067, 0.29987439, 0.31407786, 0.35138967, 0.28488771,
       0.34507524, 0.30787256, 0.31740272, 0.36625021,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.31917551, 0.30011159, 0.35157636, 0.31710593, 0.32842069,
       0.35505462, 0.31558297, 0.31387792, 0.3335018 , 0.30570826,
       0.59257806, 0.34168643, 0.32413875, 0.29428824, 0.33502138,
       0.35769511, 0.34203088, 0.2152808 , 0.29507462, 0.31899583,
       0.30046668, 0.29231127, 0.30188552, 0.30552902, 0.3024765 ,
       0.28166473, 0.29526254, 0.31804549, 0.301349  , 0.34536122,
       0.31558886, 0.29846974])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.2994063 , 0.2837668 , 0.28401872, 0.35037031, 0.30268793,
       0.43750745, 0.30589713, 0.34572127, 0.26477419, 0.2792372 ,
       0.27159065, 0.29688233, 0.30866018, 0.30541112, 0.33503147,
       0.21631556, 0.2668347 , 0.32235939, 0.2993878 ,

...

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.34224062, 0.32373967, 0.29122261, 0.31818437, 0.33139352,
       0.31185901, 0.46023972, 0.2856951 , 0.30164197, 0.30134452,
       0.31124546, 0.28628107, 0.35234237, 0.32748971, 0.29818205,
       0.31876783, 0.37239903, 0.26369132, 0.30599638, 0.37468497,
       0.32552544, 0.37836076, 0.61414265, 0.33745463, 0.29503382,
       0.30437178, 0.35010008, 0.36943187, 0.34377636, 0.21744495,
       0.35655419, 0.29389211])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.3128184 , 0.38474959, 0.33225764, 0.29103772, 0.292778  ,
       0.27471247, 0.36754193, 0.60055598, 0.309058  , 0.36020395,
       0.27775219, 0.30219322, 0.30449875, 0.30066005, 0.36804765,
       0.39563049, 0.32207003, 0.31778559, 0.2735308 ,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.43989354, 0.28235803, 0.33311782, 0.30494158, 0.30065774,
       0.30534939, 0.30022876, 0.45686573, 0.31231015, 0.3245305 ,
       0.34215682, 0.29439088, 0.29720031, 0.29035319, 0.30103038,
       0.30651858, 0.3363781 , 0.30172789, 0.39059418, 0.29770832,
       0.28659905, 0.29322081, 0.32640369, 0.35497801, 0.32856397,
       0.33371771, 0.30643155, 0.31346965, 0.30188692, 0.62085244,
       0.34756269, 0.39729232])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.30043696, 0.31651113, 0.35088854, 0.38205102, 0.37398569,
       0.34180678, 0.34871121, 0.33692843, 0.32435544, 0.32564317,
       0.41450463, 0.31201912, 0.31097758, 0.36363315, 0.19272714,
       0.30386432, 0.29994376, 0.30861413, 0.41401196,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.30237612, 0.28476534, 0.35922397, 0.21440271, 0.2841956 ,
       0.25021803, 0.29203815, 0.30239673, 0.3287857 , 0.10705456,
       0.30482559, 0.37213361, 0.36230557, 0.22057101, 0.29248598,
       0.4094305 , 0.27139914, 0.29017413, 0.34288881, 0.36542769,
       0.29793641, 0.39495877, 0.3599522 , 0.31617882, 0.30241236,
       0.41544676, 0.22263708, 0.29468205, 0.59582535, 0.36129433,
       0.34502344, 0.3324334 ])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.32710484, 0.30352737, 0.32985835, 0.30073104, 0.30663407,
       0.33597321, 0.37275861, 0.29864466, 0.33270795, 0.29521623,
       0.29137831, 0.28196323, 0.29800985, 0.28340754, 0.30439755,
       0.35323517, 0.58137261, 0.30475301, 0.28976742,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.34436833, 0.37758863, 0.3003687 , 0.28880167, 0.292983  ,
       0.29965604, 0.27662847, 0.35019375, 0.36395223, 0.32054425,
       0.24456871, 0.32328854, 0.33301608, 0.30841203, 0.49986594,
       0.30030618, 0.4120133 , 0.29301446, 0.26824304, 0.3025267 ,
       0.29222703, 0.29321162, 0.35394317, 0.36475208, 0.32203454,
       0.40693634, 0.35564808, 0.30548118, 0.16363407, 0.34133749,
       0.34458966, 0.22290121])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.33498953, 0.3566245 , 0.30936085, 0.30825143, 0.3206779 ,
       0.39090932, 0.30553463, 0.33169392, 0.30605825, 0.3057269 ,
       0.28497351, 0.3033087 , 0.28858954, 0.47541283, 0.29629341,
       0.30220389, 0.30630363, 0.34612611, 0.30956958,

(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.34846593, 0.31813255, 0.39156079, 0.29219046, 0.04550185,
       0.27837984, 0.31968621, 0.31589748, 0.3012767 , 0.3524124 ,
       0.6110728 , 0.27616293, 0.34469571, 0.32165779, 0.30438428,
       0.29986859, 0.        , 0.29967683, 0.31198343, 0.35742993,
       0.28673517, 0.3180725 , 0.35428957, 0.30859795, 0.31766391,
       0.2845339 , 0.30538704, 0.30421578, 0.30786079, 0.29098961,
       0.30043091, 0.2926054 ])>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>)
(<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([0.07501305, 0.37887261, 0.30629075, 0.30365897, 0.35511653,
       0.30740053, 0.0285192 , 0.28161197, 0.40662797, 0.3086399 ,
       0.288177  , 0.34802308, 0.31479343, 0.2843064 , 0.35739899,
       0.30917929, 0.35721223, 0.22270034, 0.3863986 ,

In [180]:
guess= np.array([['txt',1],['txt',2], ['txt', 1],['txt',3],
                 ['txt', 4], ['txt', 4]])

In [181]:
guess= np.array([['txt',1],['txt',2],['txt',3],
                 ['txt', 4]])

In [182]:
guess.shape

(4, 2)

In [183]:
guess

array([['txt', '1'],
       ['txt', '2'],
       ['txt', '3'],
       ['txt', '4']], dtype='<U3')

In [184]:
# np.unique(guess, axis=0, return_index = True,return_inverse=True, return_counts=True )

In [185]:
unique, unique_inverse, unique_counts= np.unique(guess, axis=0,return_inverse=True, return_counts=True )

In [186]:
print(unique, unique_inverse, unique_counts)

[['txt' '1']
 ['txt' '2']
 ['txt' '3']
 ['txt' '4']] [0 1 2 3] [1 1 1 1]


In [187]:
unique_counts

array([1, 1, 1, 1])

In [173]:
unique_counts.shape[0]

4

In [174]:
np.ones(unique_counts.shape[0], dtype=int)

array([1, 1, 1, 1])

In [175]:
np.where(unique_counts>1)

(array([], dtype=int64),)

In [176]:
len(np.where(unique_counts>1)[0])

0

In [177]:
if 0 == len(np.where(unique_counts>1)[0]):
    print('Here')

Here


In [137]:
unique_counts is np.ones(unique_counts.shape[0], dtype=int)


False

In [105]:
guess= np.array([[2,1],[2,2], [2, 1]])

In [39]:
guess

array([[2, 1],
       [2, 2],
       [2, 1]])

In [41]:
np.unique(guess, axis=1, return_inverse=True)

(array([[1, 2],
        [2, 2],
        [1, 2]]),
 array([1, 0]))